# SQLite3

## Introduction

SQLite is a widely used, open-source relational database management system (RDBMS) that is known for its simplicity, portability, and efficiency. Unlike other database systems that rely on a separate server process, SQLite is embedded directly into applications, meaning that the entire database engine is contained within a single library, allowing for easy integration into programs. SQLite is particularly popular in environments where a full-scale database might be unnecessary, such as small to medium-sized applications, local storage in web browsers, or for development and testing purposes. For more information on SQLite, visit the website [here](https://www.sqlite.org).

This tutorial focuses on how to use SQLite3 in python using the `sqlite3` library included as a [standard python library](https://docs.python.org/3/library/sqlite3.html).

## Getting Started

In [1]:
import pandas as pd
import sqlite3

print('sqlite3 version:', sqlite3.version)

sqlite3 version: 2.6.0


You can connect to a database by passing the name of the database. If a database does not exist, a new database will be created. SQLite3 also has the ability to run in memory by passing `:memory:` in place of the database name. This can be particularily useful for testing or in any other scenario where you need a fresh database each time a script is ran to avoid errors such as when you need to run a query to create table that may already exist otherwise.

In [2]:
conn = sqlite3.connect('sqlite-demo.db')

In order to execute SQL statements and fetch results from SQL queries, you will need to use a database cursor using the established connection.

In [3]:
c = conn.cursor()

## Working with tables

You can execute sql queries by passing the query as a string.  For lengthy queries, consider using a docstring with added line breaks to increase readibility and to comply with [PEP8](https://peps.python.org/pep-0008). Since we have a new database, the cell below is creating a new table to store data for this tutorial.

In [4]:
query = 'CREATE TABLE food(name, meal, snack)'

c.execute(query)

By default, when you modify a table you need to commit your transaction to the database. This ensures data integrity and allows you to execute multiple statements before you commit all your modifications to the table as a single transaction as a layer of QA/QC. Additionally, if you need to return to a previous version, you can use `conn.rollback()`. If you need to enable autocommit for whatever reason, you can use `conn.autocommit()`, but be aware that you cannot use rollback if this is set.

In [5]:
query = '''
    INSERT INTO food VALUES (?, ?, ?)
'''

# Data to insert
data = [
    ('Bo', 'Pizza', 'Peanut Butter'),
    ('Haley', 'Steak', 'Crackers'),
    ('Rizzo', 'Salmon', 'Dog Treats')
]

# Execute the prepared statement with the data
c.executemany(query, data)
conn.commit()

#NOTE: The above query operates the same as below, but with an added security measure. More details below.
# query = '''
#         INSERT INTO food VALUES
#         ('Bo', 'Pizza', 'Peanut Butter'),
#           ('Haley', 'Steak', 'Crackers'),
#           ('Rizzo', 'Salmon', 'Dog Treats')
# '''

# c.execute(query)

Note that above is storing the SQL query in a variable and using `?` placeholders for new values to updated, storing the new values in a seperate list of tuple variable, and using a `c.executemany()` to run the query many times. This is called a prepared statement and is one of many ways to help protect against SQL injection which gives hackers access to records and allows them to execute malicious code and even delete your tables entirely. If someone were to try and escape the prepared statement it would result in an error. **It is always recommended to use prepared statements for any query involving dynamic input.** For more information about how to protect against SQL injection, see this [link](https://cheatsheetseries.owasp.org/cheatsheets/SQL_Injection_Prevention_Cheat_Sheet.html).

To view a table, you have 3 options: `fetchone()` returns 1 result in a tuple, `fetchmany(n)` returns *n* number of results in a nested list, and `fetchall()` returns everything in a nested list.

In [6]:
query = 'SELECT * FROM food'

res = c.execute()
res.fetchall()

[('Bo', 'Pizza', 'Peanut Butter'),
 ('Haley', 'Steak', 'Crackers'),
 ('Rizzo', 'Salmon', 'Dog Treats')]

This also works in reverse if you need to write a pandas dataframes into a sqlite3 table.

In [7]:
dic = {
    'name': ['Bo', 'Haley', 'Rizzo'],
    'college': ['Georgia Tech', 'SCAD', 'Bark University'],
    'major': ['Analytics', 'Art Direction', 'Chasing Cats']
}
df = pd.DataFrame(dic)

# Write the DataFrame to a table named 'school' and replace any like-named table with the pandas one
df.to_sql('school', conn, if_exists='replace', index=False)

3

You can also read sqlite tables into a pandas dataframe.

In [8]:
query = '''
SELECT food.name, food.meal, food.snack, school.college, school.major
FROM food
JOIN school ON food.name = school.name
'''

pd.read_sql(query, conn)

,name,meal,snack,college,major
0,Bo,Pizza,Peanut Butter,Georgia Tech,Analytics
1,Haley,Steak,Crackers,SCAD,Art Direction
2,Rizzo,Salmon,Dog Treats,Bark University,Chasing Cats


## Wrapping Up

Finally, when you're done with the database, you need to close your sql connection to free up memory and prevent any database locks.

In [9]:
conn.close()